In [1]:
# | echo: true
# | output: false

import polars as pl
import seaborn.objects as so
from tabulate import tabulate

root = "/Volumes/T7/BrainLat/"
records_path = f"{root}/BrainLat_records.csv"
only_eeg = pl.scan_csv(records_path).filter(pl.col("eeg") == 1.0).collect()
display(only_eeg)

id,diagnosis,T1,Rest,DWI,MF,eeg
str,str,i64,i64,i64,f64,i64
"""PIB00002""","""CN""",0,0,0,null,1
"""PIB00003""","""CN""",1,1,1,3.0,1
"""PIB00004""","""CN""",1,1,1,3.0,1
"""PIB00006""","""CN""",1,1,1,3.0,1
"""PIB00007""","""CN""",1,1,1,3.0,1
"""PIB00008""","""CN""",1,1,1,3.0,1
"""PIB00010""","""CN""",1,1,1,3.0,1
"""PIB00012""","""CN""",1,1,1,3.0,1
"""PIB00014""","""CN""",1,1,1,3.0,1


In [2]:
# | output: false
counts = only_eeg.select(["diagnosis", "eeg"]).group_by(
    "diagnosis").agg(pl.all().sum())

cognition_path = f"{root}/Demographic and cognition/BrainLat_Cognition.csv"
cognition = (
    pl.scan_csv(cognition_path)
    .filter((pl.col("id").is_in(only_eeg.get_column("id").to_list())))
    .collect()
)
demographics_path = (
    "/Volumes/T7/BrainLat/Demographic and cognition/BrainLat_Demographic.csv"
)
demographics = (
    pl.scan_csv(demographics_path, dtypes={"Age": float})
    .filter((pl.col("id").is_in(only_eeg.get_column("id").to_list())))
    .collect()
    .with_columns(pl.col("Age").cast(pl.Int64, strict=False))
)
display(counts)
display(demographics)
display(cognition)

diagnosis,eeg
str,i64
"""MS""",32
"""CN""",42
"""FTD""",16
"""AD""",35
"""PD""",27


id,diagnosis,sex,Age,years_education,laterality
str,str,i64,i64,i64,i64
"""PIB00002""","""CN""",0,69,15,1
"""PIB00003""","""CN""",0,74,15,1
"""PIB00004""","""CN""",0,68,15,1
"""PIB00006""","""CN""",1,58,15,1
"""PIB00007""","""CN""",0,57,14,1
"""PIB00008""","""CN""",0,77,18,1
"""PIB00010""","""CN""",1,74,12,1
"""PIB00012""","""CN""",0,70,12,0
"""PIB00014""","""CN""",0,61,15,1


id,diagnosis,moca_total,moca_visuospatial,moca_recog,moca_attention,moca_language,moca_abstraction,moca_memory,moca_orientation,ifs_total_score,ifs_motor_series,ifs_conflicting_instructions,ifs_motor_inhibition,ifs_digits,ifs_months,ifs_visual_wm,ifs_proverb,ifs_verbal_inhibition,mini_sea_fer,mini_sea_tom,emotion recog
str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str
"""PIB00002""","""CN""",26,5,3,6,2,2,2,6,25,3,3,3,4,2,2,3,5,14.1,9.8,null
"""PIB00003""","""CN""",28,5,3,6,3,2,3,6,24,3,3,3,3,2,2,3,5,11.6,13.9,null
"""PIB00004""","""CN""",28,5,3,6,3,2,3,6,24,3,3,3,3,2,2,3,5,13.7,10.5,null
"""PIB00006""","""CN""",25,5,3,6,2,2,1,6,28,3,3,3,4,2,4,3,6,12.4,15.0,null
"""PIB00007""","""CN""",23,5,3,5,3,1,0,6,23,3,3,2,3,2,3,3,4,12.0,12.0,null
"""PIB00008""","""CN""",29,5,3,6,3,2,4,6,24,3,3,3,3,2,2,3,5,12.9,15.0,null
"""PIB00010""","""CN""",18,5,3,3,1,1,0,5,20,3,3,2,2,1,4,1,4,7.7,10.1,null
"""PIB00012""","""CN""",29,5,3,6,3,2,4,6,25,3,3,3,4,2,2,3,5,14.1,14.3,null
"""PIB00014""","""CN""",26,5,3,6,2,2,2,6,25,3,3,3,3,2,3,3,5,12.9,13.9,null


In [3]:
# | output: false

cognition_summary = cognition.group_by("diagnosis").agg(pl.all().count())
display(cognition_summary)

diagnosis,id,moca_total,moca_visuospatial,moca_recog,moca_attention,moca_language,moca_abstraction,moca_memory,moca_orientation,ifs_total_score,ifs_motor_series,ifs_conflicting_instructions,ifs_motor_inhibition,ifs_digits,ifs_months,ifs_visual_wm,ifs_proverb,ifs_verbal_inhibition,mini_sea_fer,mini_sea_tom,emotion recog
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""CN""",42,30,30,30,30,30,30,30,30,27,24,24,23,23,23,23,23,25,29,29,0
"""MS""",32,0,0,0,0,0,0,0,0,28,28,0,0,0,0,0,0,0,0,0,0
"""PD""",27,26,19,19,26,26,26,26,26,25,17,17,16,7,0,24,7,0,25,24,0
"""AD""",35,29,29,29,29,29,29,29,29,33,31,30,31,31,31,31,31,31,31,31,0
"""FTD""",16,14,14,14,14,14,14,14,14,14,14,14,14,14,13,14,14,16,16,16,0


In [4]:
df = cognition_summary.to_pandas().T
df.columns = df.iloc[0]
df = df.iloc[1:]
print(tabulate(df, headers="keys", tablefmt="grid"))

+------------------------------+------+------+------+------+-------+
|                              |   CN |   MS |   PD |   AD |   FTD |
+==============================+======+======+======+======+=======+
| id                           |   42 |   32 |   27 |   35 |    16 |
+------------------------------+------+------+------+------+-------+
| moca_total                   |   30 |    0 |   26 |   29 |    14 |
+------------------------------+------+------+------+------+-------+
| moca_visuospatial            |   30 |    0 |   19 |   29 |    14 |
+------------------------------+------+------+------+------+-------+
| moca_recog                   |   30 |    0 |   19 |   29 |    14 |
+------------------------------+------+------+------+------+-------+
| moca_attention               |   30 |    0 |   26 |   29 |    14 |
+------------------------------+------+------+------+------+-------+
| moca_language                |   30 |    0 |   26 |   29 |    14 |
+------------------------------+--

First lets have 2 models first model that is used only on behavioral variables, then the second we use only on the people who have a eeg data


In [5]:
# | output: false

cognition_path = "/Volumes/T7/BrainLat/Demographic and cognition/BrainLat_Cognition.csv"
cognition_all = (
    pl.scan_csv(cognition_path)
    .collect()
    .filter(~pl.all_horizontal(pl.all().is_null()))
    .group_by("diagnosis")
    .agg(pl.all().count())
)
display(cognition_all)

diagnosis,id,moca_total,moca_visuospatial,moca_recog,moca_attention,moca_language,moca_abstraction,moca_memory,moca_orientation,ifs_total_score,ifs_motor_series,ifs_conflicting_instructions,ifs_motor_inhibition,ifs_digits,ifs_months,ifs_visual_wm,ifs_proverb,ifs_verbal_inhibition,mini_sea_fer,mini_sea_tom,emotion recog
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""CN""",250,156,154,154,154,154,154,154,154,166,162,162,161,161,161,161,161,169,113,122,0
"""MS""",34,0,0,0,0,0,0,0,0,29,29,0,0,0,0,0,0,0,0,0,0
"""FTD""",165,94,95,95,95,95,95,95,95,115,112,112,112,112,112,113,113,119,59,59,0
"""AD""",279,111,111,111,111,111,111,111,111,177,175,174,175,176,176,176,176,182,169,162,0
"""PD""",56,55,20,20,33,33,33,33,33,54,18,18,17,13,0,53,13,0,32,31,22


In [6]:
df_all = cognition_all.to_pandas().T
df_all.columns = df_all.iloc[0]
df_all = df_all.iloc[1:]
print(tabulate(df_all, headers="keys", tablefmt="grid"))

+------------------------------+------+------+-------+------+------+
|                              |   CN |   MS |   FTD |   AD |   PD |
+==============================+======+======+=======+======+======+
| id                           |  250 |   34 |   165 |  279 |   56 |
+------------------------------+------+------+-------+------+------+
| moca_total                   |  156 |    0 |    94 |  111 |   55 |
+------------------------------+------+------+-------+------+------+
| moca_visuospatial            |  154 |    0 |    95 |  111 |   20 |
+------------------------------+------+------+-------+------+------+
| moca_recog                   |  154 |    0 |    95 |  111 |   20 |
+------------------------------+------+------+-------+------+------+
| moca_attention               |  154 |    0 |    95 |  111 |   33 |
+------------------------------+------+------+-------+------+------+
| moca_language                |  154 |    0 |    95 |  111 |   33 |
+------------------------------+--